# Deep Learning Assignment 2 #

#### using pytorch models to identify snow vs no snow in images, the goal of this assignment is to not only achieve higher accuracy but to also keep the number of parameters to a minimum so that results are computed as fast as possible ####

In [10]:
# imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from torchvision import transforms

## Dataloaders ##

### Dataloader for stacking ###

In [17]:
class stacking_dataloader(Dataset):
    def __init__(self, data_dir ):
        self.data_dir = data_dir
        self.all_classes = ['NotSnow','Snow']
        self.class_indexes = {cls: idx for idx, cls in enumerate(self.all_classes)}
        self.dataset = self.dataset_maker()
        self.length = len(self.dataset)
    def dataset_maker(self): # this will compile all the folder paths and the corresponding classes in a tuple to be stored in the class
        data_list = []
        for class_name in self.all_classes:
            all_class_folders =os.path.join(self.data_dir, class_name)
            for fold in os.listdir(all_class_folders):
                sample_path_n_class = os.path.join(all_class_folders, fold)
                sample = (sample_path_n_class, self.class_indexes[class_name])
                data_list.append(sample)
        return data_list
    def __len__(self):
        return self.length
    
    def __getitem__(self, index):
        sample_path, sample_class = self.dataset[index]
        # this path will have the final two images which will have to be processed individually and returned as a stacked vector for the stacking model input which we will have to consider in this case
        # all images in the dataset are named 0.png or 1.png in this case which we will hardcode in this class       
        pic1 = transforms.ToTensor()(Image.open(os.path.join(sample_path, '0.png'))) # image 1 in tensor
        pic2 = transforms.ToTensor()(Image.open(os.path.join(sample_path, '0.png'))) # image 2 in tensor
        sample = torch.stack([pic1, pic2], dim = 0) # dim = 0 because we have to stack them on top of each other in this case
        # do we need to normalise this ?
        #defining a resizing and normalisation transformation to 
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.Normalize(mean=[0.485], std=[0.5]) # using dummy values in this case for the greyscale normalisation
        ])

        # Apply the transformation pipeline to the input sample
        sample = transform(sample)
        return sample ,  sample_class

## Models ##

### Stacking Model ###

In [18]:
stacking_dataset = stacking_dataloader('./dataset/')

In [19]:
print(f"the total dataset size is {len(stacking_dataset)}")
print(f'the object returned is a tuple containing the tensor and the class label')
print(f'the shape of each tensor in this case is {stacking_dataset[1][0].shape}')

the total dataset size is 3568
the object returned is a tuple containing the tensor and the class label
the shape of each tensor in this case is torch.Size([2, 3, 224, 224])
